In [10]:
import random
import time
import pandas as pd
import requests
from datetime import datetime

class BehaviorDataGenerator:
    def __init__(self, api_base_url='http://localhost:8080/api'):
        self.api_base_url = api_base_url
        self.session = requests.Session()
        self.recipes_df = None
        self.popular_ids = []
        # Liste de commentaires réalistes pour varier les données
        self.comment_pool = [
            "Super recette, très facile à faire !",
            "Toute la famille a adoré, merci.",
            "Un peu trop salé à mon goût, mais délicieux.",
            "Parfait pour un dîner rapide le soir.",
            "J'ai ajouté un peu d'épices, c'était top !",
            "La cuisson était parfaite.",
            "Je recommande vivement cette recette."
        ]

    def login(self, email, password):
        try:
            # Note: Si votre auth est aussi sans /v1, modifiez ici
            res = self.session.post(f"{self.api_base_url}/v1/auth/login", 
                                    json={'email': email, 'motDePasse': password})
            res.raise_for_status()
            token = res.json().get('token')
            self.session.headers.update({'Authorization': f'Bearer {token}'})
            print(f"✅ Authentifié en tant que {email}")
            return True
        except Exception as e:
            print(f"❌ Erreur login: {e}")
            return False

    def load_local_data(self, csv_file):
        # 1. Chargement avec détection automatique du séparateur
        try:
            self.recipes_df = pd.read_csv(csv_file, sep=None, engine='python')
        except:
            self.recipes_df = pd.read_csv(csv_file)
        
        # 2. Nettoyage immédiat des noms de colonnes (espaces invisibles)
        self.recipes_df.columns = [str(c).strip() for c in self.recipes_df.columns]
        
        # 3. Mapping ultra-flexible
        mapping = {
            'id': ['id', 'ID', 'recette_id', 'Unnamed: 0', 'index'],
            'pays': ['pays', 'cuisine', 'Cuisine', 'Pays', 'origin'],
            'vegetarien': ['vegetarien', 'vege', 'is_vege', 'Végétarien']
        }
        
        for target, possibles in mapping.items():
            for col in self.recipes_df.columns:
                if col.lower() in [p.lower() for p in possibles]:
                    self.recipes_df = self.recipes_df.rename(columns={col: target})
                    break

        # 4. Si 'id' n'est toujours pas là, on prend la 1ère colonne ou l'index
        if 'id' not in self.recipes_df.columns:
            if self.recipes_df.index.name in ['id', 'ID']:
                self.recipes_df = self.recipes_df.reset_index().rename(columns={self.recipes_df.index.name: 'id'})
            else:
                print(f"⚠️ 'id' non trouvé. Renommage forcé de la colonne '{self.recipes_df.columns[0]}' en 'id'")
                self.recipes_df = self.recipes_df.rename(columns={self.recipes_df.columns[0]: 'id'})

        # 5. Création des IDs populaires
        num_popular = max(1, len(self.recipes_df) // 5)
        # On s'assure que 'id' est bien là avant le sample
        self.popular_ids = self.recipes_df['id'].dropna().sample(num_popular).tolist()
        
        print(f"📖 {len(self.recipes_df)} recettes chargées. Colonnes finales : {self.recipes_df.columns.tolist()}")

    def generate_behavior(self, start_user, end_user, interactions_per_user=20):
        if self.recipes_df is None: return print("❌ CSV non chargé.")

        for user_id in range(start_user, end_user + 1):
            profile = {
                'pays_prefere': random.choice(['Italie', 'France', 'Japon', 'Mexique']),
                'vege_only': random.random() < 0.2,
                'session_id': f"sess_{user_id}_{int(time.time())}"
            }

            potential_matches = self.recipes_df[
                (self.recipes_df['pays'] == profile['pays_prefere']) | 
                (self.recipes_df['vegetarien'] == profile['vege_only'])
            ]
            if potential_matches.empty: potential_matches = self.recipes_df

            for _ in range(interactions_per_user):
                target_id = random.choice(self.popular_ids if random.random() < 0.8 else potential_matches['id'].tolist())

                # 1. MongoDB : Consultation brute
                self._record_mongo_interaction(user_id, target_id, 'CONSULTATION', profile['session_id'])

                # 2. Logique d'engagement aléatoire
                rand = random.random()
                
                if rand < 0.15: # 15% Favoris
                    self._add_to_favorites(user_id, target_id, profile['session_id'])
                
                elif rand < 0.30: # 15% Note
                    self._add_rating(user_id, target_id, profile['session_id'])
                
                elif rand < 0.40: # 10% Commentaire
                    self._add_comment(user_id, target_id, profile['session_id'])

            if user_id % 10 == 0:
                print(f"➔ Utilisateur {user_id} traité...")

    def _add_to_favorites(self, u_id, r_id, s_id):
        status = self._post_to_api(f"/favoris/{u_id}/{r_id}")
        if status in [200, 201]:
            self._record_mongo_interaction(u_id, r_id, 'FAVORI_AJOUTE', s_id)

    def _add_rating(self, u_id, r_id, s_id):
        """Ajoute une note (MySQL) et trace l'interaction (MongoDB)"""
        valeur = random.choices([5, 4, 3, 2], weights=[40, 40, 15, 5])[0]
        # URL basée sur la logique simplifiée : /api/notes/{userId}/{recetteId}
        status = self._post_to_api(f"/notes/{u_id}/{r_id}", json_data={'valeur': valeur})
        if status in [200, 201]:
            self._record_mongo_interaction(u_id, r_id, 'NOTE_POSEE', s_id)

    def _add_comment(self, u_id, r_id, s_id):
        """Ajoute un commentaire (MySQL) et trace l'interaction (MongoDB)"""
        commentaire = random.choice(self.comment_pool)
        # URL basée sur la logique simplifiée : /api/commentaires/{userId}/{recetteId}
        status = self._post_to_api(f"/commentaires/{u_id}/{r_id}", json_data={'contenu': commentaire})
        if status in [200, 201]:
            self._record_mongo_interaction(u_id, r_id, 'COMMENTAIRE_POSTE', s_id)

    def _record_mongo_interaction(self, u_id, r_id, action, s_id):
        params = {'idUser': u_id, 'idRecette': r_id, 'typeInteraction': action, 'sessionId': s_id}
        self._post_to_api("/v1/recette-interactions/registrer", params=params)

    def _post_to_api(self, endpoint, json_data=None, params=None):
        try:
            res = self.session.post(f"{self.api_base_url}{endpoint}", json=json_data, params=params, timeout=5)
            return res.status_code
        except:
            return None

if __name__ == "__main__":
    gen = BehaviorDataGenerator(api_base_url='http://localhost:8080/api')
    if gen.login('dianekassi@admin.com', 'Mydayana48'):
        gen.load_local_data('recettes_clean.csv')
        # Plage d'utilisateurs à simuler
        gen.generate_behavior(2001, 2030, interactions_per_user=15)
        print("\n✨ Simulation complète terminée (Favoris, Notes, Commentaires).")

✅ Authentifié en tant que dianekassi@admin.com
⚠️ 'id' non trouvé. Renommage forcé de la colonne 'url' en 'id'
📖 288 recettes chargées. Colonnes finales : ['id', 'titre', 'description', 'difficulte', 'budget', 'temps_preparation', 'temps_cuisson', 'type_recette', 'pays', 'ingredients', 'etapes', 'date_scraping', 'imageUrl', 'categorie', 'vegetarien']
➔ Utilisateur 2010 traité...
➔ Utilisateur 2020 traité...
➔ Utilisateur 2030 traité...

✨ Simulation complète terminée (Favoris, Notes, Commentaires).
